In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
import scipy as sp
from collections import defaultdict
from itertools import product
import gurobipy as gb
from gurobipy import GRB

repeats = json.load(open('../cleanerdata/repeats.json'))
repeats = {int(k):v for k,v in repeats.items()}

locdem = pd.read_excel('../cleanerdata/locdem.xlsx')

q = locdem['Number of pallets'].to_numpy().astype(float).tolist()
q = [0] + q
q.extend([v['dem'] for v in repeats.values()])

locs = pd.read_excel('../cleanerdata/locations.xlsx')
longs, lats = locs['long'].to_numpy().tolist(), locs['lat'].to_numpy().tolist()
longs.extend([longs[v['map']] for v in repeats.values()])
lats.extend([lats[v['map']] for v in repeats.values()])

distmat = pd.read_json('../cleanerdata/distmat.json').to_numpy()
timemat = pd.read_json('../cleanerdata/timemat.json').to_numpy()
def get(i):
    try:
        i = repeats[i]['map']
    except KeyError:
        pass
    return i

def cost(i,j):
    i,j = get(i), get(j)
    return distmat[i,j]
def time(i,j):
    i,j = get(i), get(j)
    return timemat[i,j]

def location(i):
    return longs[i], lats[i]

In [2]:
loops = json.load(open('./loops.json'))
loops

[[0, 105, 67, 1, 45, 0],
 [0, 166, 0],
 [0, 28, 93, 2, 66, 123, 0],
 [0, 69, 120, 61, 60, 3, 0],
 [0, 4, 40, 0],
 [0, 151, 0],
 [0, 152, 0],
 [0, 153, 0],
 [0, 5, 0],
 [0, 25, 125, 0],
 [0, 126, 0],
 [0, 127, 0],
 [0, 89, 103, 6, 0],
 [0, 128, 0],
 [0, 130, 0],
 [0, 59, 0],
 [0, 102, 47, 7, 16, 0],
 [0, 43, 8, 27, 0],
 [0, 160, 0],
 [0, 162, 0],
 [0, 9, 32, 0],
 [0, 131, 133, 0],
 [0, 134, 0],
 [0, 155, 117, 0],
 [0, 156, 62, 10, 0],
 [0, 84, 75, 171, 11, 51, 0],
 [0, 12, 0],
 [0, 110, 81, 88, 132, 0],
 [0, 99, 13, 74, 20, 77, 0],
 [0, 161, 78, 113, 14, 34, 0],
 [0, 15, 79, 104, 0],
 [0, 55, 165, 0],
 [0, 146, 50, 17, 56, 63, 0],
 [0, 18, 173, 58, 101, 44, 0],
 [0, 119, 87, 19, 95, 0],
 [0, 178, 0],
 [0, 21, 122, 176, 57, 0],
 [0, 180, 0],
 [0, 106, 0],
 [0, 167, 115, 26, 22, 0],
 [0, 121, 76, 23, 100, 0],
 [0, 179, 0],
 [0, 24, 107, 82, 0],
 [0, 168, 135, 0],
 [0, 42, 94, 53, 29, 54, 0],
 [0, 91, 30, 83, 80, 90, 0],
 [0, 31, 33, 0],
 [0, 136, 0],
 [0, 137, 0],
 [0, 138, 0],
 [0, 35, 1

In [3]:
h = np.zeros(len(loops), dtype=float)
for i, loop in enumerate(loops):
    for j in range(0, len(loop)-1):
        arc = loop[j], loop[j+1]
        h[i] += time(*arc)
h = h / 3600

In [4]:
D = range(16)
L = range(0, len(loops))
T = range(5)

m = gb.Model()
z = m.addVars(D, vtype=GRB.BINARY, name='z')
y = m.addVars(L,T,D,vtype=GRB.BINARY, name='y')

m.setObjective(z.sum('*'), GRB.MINIMIZE)
'''for l in L:
    expr = 0
    for t in T:
        for d in D:
            expr += y[l,t,d]
    m.addConstr(expr == 1)'''
    
m.addConstrs((y.sum(l,'*', '*') == 1 for l in L))
m.addConstrs((
    sum([h[l]*y[l,t,d] for l in L]) <= 8 for t in T for d in D
))
m.addConstrs((y[l,t,d] <= z[d] for l in L for t in T for d in D))
m.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-12
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6317 rows, 6176 columns and 24640 nonzeros
Model fingerprint: 0x6ed7a69e
Variable types: 0 continuous, 6176 integer (6176 binary)
Coefficient statistics:
  Matrix range     [3e-01, 4e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 16.0000000
Presolve time: 0.07s
Presolved: 6317 rows, 6176 columns, 24640 nonzeros
Variable types: 0 continuous, 6176 integer (6176 binary)

Root relaxation: objective 2.000000e-01, 6170 iterations, 0.24 seconds (0.06 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.20000    0 1158   16

In [5]:
Loops = []
for loop in loops:
    L = []
    for l in loop:
        if l in repeats:
            L.append(repeats[l]['map'])
        else:
            L.append(l)
    Loops.append(L)

In [6]:
Loops

[[0, 105, 67, 1, 45, 0],
 [0, 105, 0],
 [0, 28, 93, 2, 66, 123, 0],
 [0, 69, 120, 61, 60, 3, 0],
 [0, 4, 40, 0],
 [0, 40, 0],
 [0, 40, 0],
 [0, 40, 0],
 [0, 5, 0],
 [0, 25, 5, 0],
 [0, 5, 0],
 [0, 5, 0],
 [0, 89, 103, 6, 0],
 [0, 6, 0],
 [0, 6, 0],
 [0, 59, 0],
 [0, 102, 47, 7, 16, 0],
 [0, 43, 8, 27, 0],
 [0, 43, 0],
 [0, 43, 0],
 [0, 9, 32, 0],
 [0, 9, 32, 0],
 [0, 32, 0],
 [0, 41, 117, 0],
 [0, 41, 62, 10, 0],
 [0, 84, 75, 110, 11, 51, 0],
 [0, 12, 0],
 [0, 110, 81, 88, 12, 0],
 [0, 99, 13, 74, 20, 77, 0],
 [0, 43, 78, 113, 14, 34, 0],
 [0, 15, 79, 104, 0],
 [0, 55, 104, 0],
 [0, 38, 50, 17, 56, 63, 0],
 [0, 18, 112, 58, 101, 44, 0],
 [0, 119, 87, 19, 95, 0],
 [0, 119, 0],
 [0, 21, 122, 117, 57, 0],
 [0, 122, 0],
 [0, 106, 0],
 [0, 106, 115, 26, 22, 0],
 [0, 121, 76, 23, 100, 0],
 [0, 121, 0],
 [0, 24, 107, 82, 0],
 [0, 107, 32, 0],
 [0, 42, 94, 53, 29, 54, 0],
 [0, 91, 30, 83, 80, 90, 0],
 [0, 31, 33, 0],
 [0, 33, 0],
 [0, 33, 0],
 [0, 33, 0],
 [0, 35, 108, 6, 86, 0],
 [0, 108, 0],

In [7]:
h

array([2.13888889, 1.05583333, 3.59555556, 2.68111111, 1.15277778,
       1.145     , 1.145     , 1.145     , 0.96027778, 2.83305556,
       0.96027778, 0.96027778, 2.27138889, 1.17944444, 1.17944444,
       0.30972222, 0.82972222, 3.73916667, 0.99694444, 0.99694444,
       1.10583333, 1.10583333, 1.07      , 0.90916667, 1.87888889,
       1.90694444, 1.82194444, 2.11777778, 2.92      , 1.27638889,
       0.87833333, 0.85472222, 2.11916667, 1.95083333, 1.71361111,
       1.12444444, 2.17861111, 0.75027778, 1.03166667, 1.16833333,
       3.01305556, 1.21194444, 1.25777778, 1.14083333, 1.91361111,
       2.68861111, 1.36194444, 1.25694444, 1.25694444, 1.25694444,
       1.75388889, 1.16944444, 0.89944444, 1.13333333, 0.89944444,
       0.925     , 0.89972222, 0.925     , 0.87611111, 0.87611111,
       0.87333333, 1.18444444, 1.31555556, 1.18444444, 1.18444444,
       2.13888889, 2.15972222, 1.03888889, 3.07777778, 3.07777778,
       3.26916667, 2.48416667, 1.08638889, 1.10472222, 1.07861

In [8]:
soln = json.loads(m.getJSONSolution())

In [9]:
soln

{'SolutionInfo': {'Status': 2,
  'Runtime': 8.067200899124146,
  'Work': 3.2966673376940787,
  'ObjVal': 3,
  'ObjBound': 3,
  'ObjBoundC': 3,
  'MIPGap': 0,
  'IntVio': 0,
  'BoundVio': 0,
  'ConstrVio': 0,
  'IterCount': 20556,
  'BarIterCount': 0,
  'NodeCount': 3,
  'SolCount': 5,
  'PoolObjBound': 3,
  'PoolObjVal': [3, 4, 6, 15, 16]},
 'Vars': [{'VarName': 'z[0]', 'X': 1},
  {'VarName': 'z[10]', 'X': 1},
  {'VarName': 'z[12]', 'X': 1},
  {'VarName': 'y[0,0,0]', 'X': 1},
  {'VarName': 'y[1,2,10]', 'X': 1},
  {'VarName': 'y[2,0,0]', 'X': 1},
  {'VarName': 'y[3,2,0]', 'X': 1},
  {'VarName': 'y[4,0,0]', 'X': 1},
  {'VarName': 'y[5,1,10]', 'X': 1},
  {'VarName': 'y[6,0,10]', 'X': 1},
  {'VarName': 'y[7,0,10]', 'X': 1},
  {'VarName': 'y[8,0,0]', 'X': 1},
  {'VarName': 'y[9,4,0]', 'X': 1},
  {'VarName': 'y[10,3,10]', 'X': 1},
  {'VarName': 'y[11,3,10]', 'X': 1},
  {'VarName': 'y[12,2,0]', 'X': 1},
  {'VarName': 'y[13,0,10]', 'X': 1},
  {'VarName': 'y[14,0,10]', 'X': 1},
  {'VarName': 'y